In [1]:
import hopsworks
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns
from matplotlib import pyplot
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
import joblib
import os

/home/redve/Kurser Master/ID2223 Scalable Machine Learning and Deep Learning/ID2223/Lab1/Task2/venv/lib64/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
for k, v in os.environ.items():
    if "HOPS" in k:
        print(f'{k}={v}')

In [4]:
# You have to set the environment variable 'HOPSWORKS_API_KEY' for login to succeed
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/190613
Connected. Call `.close()` to terminate connection gracefully.


In [6]:
bostadfg1 = fs.get_feature_group(name="bostad", version=4)
query1 = bostadfg1.select_all()
feature_view1 = fs.get_or_create_feature_view(name="bostad",
                                             version=4,
                                             description="bostad",
                                             labels=["sold_price_per_million"],
                                             query=query1)


In [7]:
x_train, x_test, y_train, y_test = feature_view1.train_test_split(test_size=0.2)

Finished: Reading data from Hopsworks, using ArrowFlight (4.79s) 


In [8]:
pca = joblib.load("pca.pkl")

x_train = pca.transform(x_train)
x_test = pca.transform(x_test)


In [9]:
!pip install tensorflow==2.10.1

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement tensorflow==2.10.1 (from versions: none)
ERROR: No matching distribution found for tensorflow==2.10.1


In [11]:
# neural networks with keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import optimizers
from keras import regularizers
from keras import initializers
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold


def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(100, input_dim=13, kernel_initializer='normal', activation='relu'))  # Update input_dim to match the number of features
    model.add(Dense(50, kernel_initializer='normal', activation='relu'))
    model.add(Dense(25, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])
    return model

# define the model
model = baseline_model()

checkpoint = ModelCheckpoint('best_model1.h5', verbose=0, monitor='val_loss',save_best_only=True, mode='auto')

history = model.fit(x_train, y_train, epochs=2000, batch_size=150, verbose=1, validation_split=0.2, callbacks=[checkpoint])


Epoch 1/2000


86/86 [==============================] - 1s 6ms/step - loss: 17.1468 - mse: 17.1468 - mae: 2.8838 - val_loss: 1.2717 - val_mse: 1.2717 - val_mae: 0.7767
Epoch 2/2000
86/86 [==============================] - 0s 4ms/step - loss: 1.0117 - mse: 1.0117 - mae: 0.6711 - val_loss: 0.9533 - val_mse: 0.9533 - val_mae: 0.6273
Epoch 3/2000
86/86 [==============================] - 0s 4ms/step - loss: 0.8450 - mse: 0.8450 - mae: 0.6088 - val_loss: 0.8597 - val_mse: 0.8597 - val_mae: 0.5901
Epoch 4/2000
86/86 [==============================] - 0s 4ms/step - loss: 0.7735 - mse: 0.7735 - mae: 0.5754 - val_loss: 0.8035 - val_mse: 0.8035 - val_mae: 0.5608
Epoch 5/2000
86/86 [==============================] - 0s 4ms/step - loss: 0.7270 - mse: 0.7270 - mae: 0.5509 - val_loss: 0.7720 - val_mse: 0.7720 - val_mae: 0.5427
Epoch 6/2000
86/86 [==============================] - 0s 4ms/step - loss: 0.6999 - mse: 0.6999 - mae: 0.5310 - val_loss: 0.7112 - val_mse: 0.7112 - val_mae: 0.5274
Epoch 7/2000
86/86 [=======

In [12]:
scores = model.evaluate(x_test, y_test)
print("\n%s: %.2f" % (model.metrics_names[1], scores[1]))

125/125 [==============================] - 0s 2ms/step - loss: 0.5103 - mse: 0.5103 - mae: 0.4526

mse: 0.51


In [15]:
mr = project.get_model_registry()

model_dir="./best_model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)
joblib.dump(model, model_dir + "/best_model.h5")

input_schema = Schema(x_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema, output_schema)

propertyModel = mr.python.create_model(
    name="property_model", 
    model_schema=model_schema,
    description="property_model",
    version=3,
)


propertyModel.save(model_dir)


Connected. Call `.close()` to terminate connection gracefully.
2024-01-08 12:51:25,319 INFO: Assets written to: ram://d8aecb68-0f1c-4290-ba98-77cc9bf83538/assets


Model export complete: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:11<00:00,  1.86s/it]

Model created, explore it at https://c.app.hopsworks.ai:443/p/190613/models/property_model/3


Model(name: 'property_model', version: 3)